In [1]:
!pip install textblob


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import library
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

In [4]:
# Import processed data
playlistDF = pd.read_csv("./processed_data.csv")
print(playlistDF.columns)
playlistDF.drop(columns=["Unnamed: 0",'Unnamed: 0.1'], inplace = True)
playlistDF.head()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'pos', 'artist_name', 'track_uri',
       'artist_uri', 'track_name', 'album_uri', 'duration_ms_x', 'album_name',
       'name', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms_y', 'time_signature', 'artist_pop', 'genres', 'track_pop'],
      dtype='object')


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,name,danceability,...,type,id,uri,track_href,analysis_url,duration_ms_y,time_signature,artist_pop,genres,track_pop
0,0,WILDES,3uvsVUrAaGQJCTEUR1S3Sx,spotify:artist:0ypTT9UqAU5sZpPo5JZmjR,Bare,spotify:album:2kIoMj5Ht14l2PnNRa2abC,242060,Bare,NewNew,0.523,...,audio_features,3uvsVUrAaGQJCTEUR1S3Sx,spotify:track:3uvsVUrAaGQJCTEUR1S3Sx,https://api.spotify.com/v1/tracks/3uvsVUrAaGQJ...,https://api.spotify.com/v1/audio-analysis/3uvs...,242060,4,32,british_singer-songwriter,0
1,1,MIIA,0heE5tAAaDQmnGhVDImPl2,spotify:artist:0h3YCmvRJ2jqt4jFiR6nGL,Dynasty,spotify:album:3g7TTE6375PGIBsM9Tlk9I,225515,Dynasty,NewNew,0.493,...,audio_features,0heE5tAAaDQmnGhVDImPl2,spotify:track:0heE5tAAaDQmnGhVDImPl2,https://api.spotify.com/v1/tracks/0heE5tAAaDQm...,https://api.spotify.com/v1/audio-analysis/0heE...,225515,4,51,norwegian_pop,65
2,2,Cold War Kids,3omXshBamrREltcf24gYDC,spotify:artist:6VDdCwrBM4qQaGxoAyxyJC,First,spotify:album:2H09itV5a5yUcGyk9u9HwY,200360,Hold My Home,NewNew,0.468,...,audio_features,3omXshBamrREltcf24gYDC,spotify:track:3omXshBamrREltcf24gYDC,https://api.spotify.com/v1/tracks/3omXshBamrRE...,https://api.spotify.com/v1/audio-analysis/3omX...,200360,4,58,indie_rock modern_alternative_rock modern_rock...,71
3,12,Cold War Kids,3omXshBamrREltcf24gYDC,spotify:artist:6VDdCwrBM4qQaGxoAyxyJC,First,spotify:album:2H09itV5a5yUcGyk9u9HwY,200360,Hold My Home,Yass,0.468,...,audio_features,3omXshBamrREltcf24gYDC,spotify:track:3omXshBamrREltcf24gYDC,https://api.spotify.com/v1/tracks/3omXshBamrRE...,https://api.spotify.com/v1/audio-analysis/3omX...,200360,4,58,indie_rock modern_alternative_rock modern_rock...,71
4,8,Cold War Kids,3omXshBamrREltcf24gYDC,spotify:artist:6VDdCwrBM4qQaGxoAyxyJC,First,spotify:album:2H09itV5a5yUcGyk9u9HwY,200360,Hold My Home,Summer '17,0.468,...,audio_features,3omXshBamrREltcf24gYDC,spotify:track:3omXshBamrREltcf24gYDC,https://api.spotify.com/v1/tracks/3omXshBamrRE...,https://api.spotify.com/v1/audio-analysis/3omX...,200360,4,58,indie_rock modern_alternative_rock modern_rock...,71


In [5]:
playlistDF[['artist_name','track_name','name']]

,artist_name,track_name,name
0,WILDES,Bare,NewNew
1,MIIA,Dynasty,NewNew
2,Cold War Kids,First,NewNew
3,Cold War Kids,First,Yass
4,Cold War Kids,First,Summer '17
...,...,...,...
68082,Cheers Elephant,Peoples,Lab
68083,Disco Fries,The Light - Radio Edit,TENnis
68084,Tove Lo,Scream My Name,TENnis
68085,Ellie Goulding,Goodness Gracious - The Chainsmokers Extended ...,TENnis


In [6]:
# Drop song duplicates
def drop_duplicates(df):
    '''
    Drop duplicate songs
    '''
    df['artists_song'] = df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return df.drop_duplicates('artists_song')

songDF = drop_duplicates(playlistDF)
print("Are all songs unique: ",len(pd.unique(songDF.artists_song))==len(songDF))


Are all songs unique:  True


In [7]:
# Select useful columns
def select_cols(df):
       '''
       Select useful columns
       '''
       return df[['artist_name','id','track_name','danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "artist_pop", "genres", "track_pop"]]
songDF = select_cols(songDF)
songDF.head()


,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop
0,WILDES,3uvsVUrAaGQJCTEUR1S3Sx,Bare,0.523,0.422,3,-10.385,0,0.0270,0.78300,0.005140,0.0898,0.145,129.876,32,british_singer-songwriter,0
1,MIIA,0heE5tAAaDQmnGhVDImPl2,Dynasty,0.493,0.594,8,-4.404,0,0.0378,0.25600,0.000000,0.0759,0.304,123.751,51,norwegian_pop,65
2,Cold War Kids,3omXshBamrREltcf24gYDC,First,0.468,0.692,2,-4.015,1,0.0295,0.02020,0.000002,0.5230,0.561,78.009,58,indie_rock modern_alternative_rock modern_rock...,71
18,Violet Days,6TYWE19e35N7Bn5heHwyY6,Your Girl,0.553,0.564,9,-7.072,0,0.0418,0.03900,0.000000,0.3180,0.310,149.953,55,indie_electropop,32
20,Drake,1xznGGDReH1oQq0xzbwXa3,One Dance,0.791,0.619,1,-5.886,1,0.0532,0.00784,0.004230,0.3510,0.371,103.989,95,canadian_hip_hop canadian_pop hip_hop pop_rap rap,40


In [8]:
def genre_preprocess(df):
    '''
    Preprocess the genre data
    '''
    df['genres_list'] = df['genres'].apply(lambda x: x.split(" "))
    return df
songDF = genre_preprocess(songDF)
songDF['genres_list'].head()


0                           [british_singer-songwriter]
1                                       [norwegian_pop]
2     [indie_rock, modern_alternative_rock, modern_r...
18                                   [indie_electropop]
20    [canadian_hip_hop, canadian_pop, hip_hop, pop_...
Name: genres_list, dtype: object

In [9]:
def playlist_preprocess(df):
    '''
    Preprocess imported playlist
    '''
    df = drop_duplicates(df)
    df = select_cols(df)
    df = genre_preprocess(df)

    return df

In [10]:
def getSubjectivity(text):
  '''
  Getting the Subjectivity using TextBlob
  '''
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  '''
  Getting the Polarity using TextBlob
  '''
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  '''
  Categorizing the Polarity & Subjectivity score
  '''
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  '''
  Perform sentiment analysis on text
  ---
  Input:
  df (pandas dataframe): Dataframe of interest
  text_col (str): column of interest
  '''
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [11]:
# Show result
sentiment = sentiment_analysis(songDF, "track_name")
sentiment.head()

,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity
0,WILDES,3uvsVUrAaGQJCTEUR1S3Sx,Bare,0.523,0.422,3,-10.385,0,0.0270,0.78300,0.005140,0.0898,0.145,129.876,32,british_singer-songwriter,0,[british_singer-songwriter],low,Positive
1,MIIA,0heE5tAAaDQmnGhVDImPl2,Dynasty,0.493,0.594,8,-4.404,0,0.0378,0.25600,0.000000,0.0759,0.304,123.751,51,norwegian_pop,65,[norwegian_pop],low,Neutral
2,Cold War Kids,3omXshBamrREltcf24gYDC,First,0.468,0.692,2,-4.015,1,0.0295,0.02020,0.000002,0.5230,0.561,78.009,58,indie_rock modern_alternative_rock modern_rock...,71,"[indie_rock, modern_alternative_rock, modern_r...",medium,Positive
18,Violet Days,6TYWE19e35N7Bn5heHwyY6,Your Girl,0.553,0.564,9,-7.072,0,0.0418,0.03900,0.000000,0.3180,0.310,149.953,55,indie_electropop,32,[indie_electropop],low,Neutral
20,Drake,1xznGGDReH1oQq0xzbwXa3,One Dance,0.791,0.619,1,-5.886,1,0.0532,0.00784,0.004230,0.3510,0.371,103.989,95,canadian_hip_hop canadian_pop hip_hop pop_rap rap,40,"[canadian_hip_hop, canadian_pop, hip_hop, pop_...",low,Neutral


In [12]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df


In [13]:
# One-hot encoding for the subjectivity 
subject_ohe = ohe_prep(sentiment, 'subjectivity','subject')
subject_ohe.iloc[0]


subject|high      False
subject|low        True
subject|medium    False
Name: 0, dtype: bool

In [14]:
# TF-IDF implementation
tfidf = TfidfVectorizer()
tfidf_matrix =  tfidf.fit_transform(songDF['genres_list'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out() ]
genre_df.drop(columns='genre|unknown')
genre_df.reset_index(drop = True, inplace=True)
genre_df.iloc[0]

genre|21st_century_classical    0.0
genre|5th_wave_emo              0.0
genre|_hip_hop                  0.0
genre|_indie                    0.0
genre|_roll                     0.0
                               ... 
genre|zenonesque                0.0
genre|zhongguo_feng             0.0
genre|zimdancehall              0.0
genre|zolo                      0.0
genre|zydeco                    0.0
Name: 0, Length: 2186, dtype: float64

In [15]:
# artist_pop distribution descriptive stats
print(songDF['artist_pop'].describe())

count    34366.000000
mean        55.296718
std         19.852848
min          0.000000
25%         44.000000
50%         58.000000
75%         70.000000
max        100.000000
Name: artist_pop, dtype: float64


In [16]:
# Normalization
pop = songDF[["artist_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
pop_scaled.head()


,artist_pop
0,0.32
1,0.51
2,0.58
3,0.55
4,0.95


In [17]:
def create_feature_set(df, float_cols):
    '''
    Process spotify df to create a final set of features that will be used to generate recommendations
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    float_cols (list(str)): List of float columns that will be scaled
            
    Output: 
    final (pandas dataframe): Final set of features 
    '''
    
    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres_list'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='genre|unknown') # drop unknown genre
    genre_df.reset_index(drop = True, inplace=True)
    
    # Sentiment analysis
    df = sentiment_analysis(df, "track_name")

    # One-hot Encoding
    subject_ohe = ohe_prep(df, 'subjectivity','subject') * 0.3
    polar_ohe = ohe_prep(df, 'polarity','polar') * 0.5
    key_ohe = ohe_prep(df, 'key','key') * 0.5
    mode_ohe = ohe_prep(df, 'mode','mode') * 0.5

    # Normalization
    # Scale popularity columns
    pop = df[["artist_pop","track_pop"]].reset_index(drop = True)
    scaler = MinMaxScaler()
    pop_scaled = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns) * 0.2 

    # Scale audio columns
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    # Concanenate all features
    final = pd.concat([genre_df, floats_scaled, pop_scaled, subject_ohe, polar_ohe, key_ohe, mode_ohe], axis = 1)
    
    # Add song id
    final['id']=df['id'].values
    
    return final


In [18]:
# Save the data and generate the features
float_cols = songDF.dtypes[songDF.dtypes == 'float64'].index.values
songDF.to_csv("./allsong_data.csv", index = False)

# Generate features
complete_feature_set = create_feature_set(songDF, float_cols=float_cols)
complete_feature_set.to_csv("./complete_feature.csv", index = False)
complete_feature_set.head()

,genre|21st_century_classical,genre|5th_wave_emo,genre|_hip_hop,genre|_indie,genre|_roll,genre|a_cappella,genre|abstract,genre|abstract_beats,genre|abstract_hip_hop,genre|abstract_idm,...,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,3uvsVUrAaGQJCTEUR1S3Sx
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0heE5tAAaDQmnGhVDImPl2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,3omXshBamrREltcf24gYDC
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,6TYWE19e35N7Bn5heHwyY6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1xznGGDReH1oQq0xzbwXa3


In [19]:
### This is the test data
# playlistDF_test = pd.read_csv("../data/test_playlist.csv")
# playlistDF_test = playlist_preprocess(playlistDF_test)
# playlistDF_test.head()

# Test playlist:  Mom's playlist
playlistDF_test = playlistDF[playlistDF['name']=="Summer '17"]
playlistDF_test.head()
playlistDF_test.to_csv("./test_playlist.csv")

In [20]:
def generate_playlist_feature(complete_feature_set, playlist_df):
    '''
    Summarize a user's playlist into a single vector
    ---
    Input: 
    complete_feature_set (pandas dataframe): Dataframe which includes all of the features for the spotify songs
    playlist_df (pandas dataframe): playlist dataframe
        
    Output: 
    complete_feature_set_playlist_final (pandas series): single vector feature that summarizes the playlist
    complete_feature_set_nonplaylist (pandas dataframe): 
    '''
    
    # Find song features in the playlist
    complete_feature_set_playlist = complete_feature_set[complete_feature_set['id'].isin(playlist_df['id'].values)]
    # Find all non-playlist song features
    complete_feature_set_nonplaylist = complete_feature_set[~complete_feature_set['id'].isin(playlist_df['id'].values)]
    complete_feature_set_playlist_final = complete_feature_set_playlist.drop(columns = "id")
    return complete_feature_set_playlist_final.sum(axis = 0), complete_feature_set_nonplaylist


In [21]:
# Generate the features
complete_feature_set_playlist_vector, complete_feature_set_nonplaylist = generate_playlist_feature(complete_feature_set, playlistDF_test)


In [22]:
# Non-playlist features
complete_feature_set_nonplaylist.head()

,genre|21st_century_classical,genre|5th_wave_emo,genre|_hip_hop,genre|_indie,genre|_roll,genre|a_cappella,genre|abstract,genre|abstract_beats,genre|abstract_hip_hop,genre|abstract_idm,...,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1,id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,3uvsVUrAaGQJCTEUR1S3Sx
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0heE5tAAaDQmnGhVDImPl2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,6TYWE19e35N7Bn5heHwyY6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1xznGGDReH1oQq0xzbwXa3
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0yy6a8NCTaaVQwyEPIi8UU


In [23]:

# Summarized playlist features
complete_feature_set_playlist_vector


genre|21st_century_classical    0.000000
genre|5th_wave_emo              0.000000
genre|_hip_hop                  0.000000
genre|_indie                    0.917475
genre|_roll                     0.000000
                                  ...   
key|9                           1.000000
key|10                          0.000000
key|11                          0.500000
mode|0                          2.000000
mode|1                          6.500000
Length: 2217, dtype: float64

In [24]:
def generate_playlist_recos(df, features, nonplaylist_features):
    '''
    Generated recommendation based on songs in aspecific playlist.
    ---
    Input: 
    df (pandas dataframe): spotify dataframe
    features (pandas series): summarized playlist feature (single vector)
    nonplaylist_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Output: 
    non_playlist_df_top_40: Top 40 recommendations for that playlist
    '''
    
    non_playlist_df = df[df['id'].isin(nonplaylist_features['id'].values)]
    # Find cosine similarity between the playlist and the complete song set
    non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]
    non_playlist_df_top_40 = non_playlist_df.sort_values('sim',ascending = False).head(40)
    
    return non_playlist_df_top_40

In [25]:
# Genreate top 10 recommendation
recommend = generate_playlist_recos(songDF, complete_feature_set_playlist_vector, complete_feature_set_nonplaylist)
recommend.head(10)

/var/folders/4z/qjrn91vn35n8msqdc6t9tdrw0000gn/T/ipykernel_15511/2729577432.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist_df['sim'] = cosine_similarity(nonplaylist_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


,artist_name,id,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,artist_pop,genres,track_pop,genres_list,subjectivity,polarity,sim
51877,Ed Sheeran,5KL4iZkCTZyXl7KnHgfVDj,"You Need Me, I Don't Need You",0.749,0.837,7,-8.837,1,0.0512,0.1620,...,0.525,0.927,103.483,86,pop singer-songwriter_pop uk_pop,63,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.800163
62291,Ed Sheeran,6SznMYPQJHzr3IwsAHLId2,Bibia Be Ye Ye,0.738,0.677,7,-5.121,1,0.0824,0.5200,...,0.133,0.882,127.067,86,pop singer-songwriter_pop uk_pop,62,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.800062
44048,Ed Sheeran,1k5Fru5iZKKlgWOQCv35kQ,Everything You Are,0.629,0.707,7,-5.504,1,0.0947,0.3620,...,0.112,0.369,173.847,86,pop singer-songwriter_pop uk_pop,39,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.793149
46583,Ed Sheeran,3Lfiu5sZ4M4B6JaKMBc0FU,Barcelona,0.747,0.760,1,-4.294,1,0.1870,0.4480,...,0.153,0.682,99.975,86,pop singer-songwriter_pop uk_pop,65,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.786998
41244,Ed Sheeran,2A17f6I4fh6GqYX0J8noGr,Galway Girl - Martin Jensen Remix,0.726,0.748,9,-4.576,1,0.0438,0.0245,...,0.141,0.731,100.055,86,pop singer-songwriter_pop uk_pop,52,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.784862
9683,Ed Sheeran,6gTJaPuj8DT8RjuDJyBgzP,Don't,0.806,0.608,1,-7.008,1,0.0659,0.0113,...,0.635,0.849,95.049,86,pop singer-songwriter_pop uk_pop,58,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.783658
44071,Ed Sheeran,2dfHh7ECGxfNqZTQno09Vk,Hearts Don't Break Around Here,0.604,0.366,7,-7.881,1,0.0267,0.7190,...,0.162,0.181,105.177,86,pop singer-songwriter_pop uk_pop,63,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.783372
56680,Ed Sheeran,3zvc53juLgo3LibcwGBDqn,Take It Back,0.754,0.613,5,-6.342,1,0.2290,0.3180,...,0.335,0.904,113.196,86,pop singer-songwriter_pop uk_pop,31,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.782349
52982,Ed Sheeran,49WNdS1zw5NjaqMNEARcWO,Don't - Rick Ross Remix,0.800,0.630,1,-7.103,1,0.1220,0.0147,...,0.559,0.822,95.046,86,pop singer-songwriter_pop uk_pop,43,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.782151
47347,Ed Sheeran,6PJ8FF6UR8FZXfEvpHkIVN,How Would You Feel (Paean),0.617,0.439,9,-5.630,1,0.0269,0.4240,...,0.127,0.242,139.979,86,pop singer-songwriter_pop uk_pop,67,"[pop, singer-songwriter_pop, uk_pop]",low,Neutral,0.776953


In [26]:
playlistDF_test[["artist_name","track_name"]][:20]

,artist_name,track_name
4,Cold War Kids,First
5810,WALLA,101
6107,Coldplay,The Scientist
16233,Empire of the Sun,Walking On A Dream
22666,Calvin Harris,Slide
26248,Charlie Puth,Attention
26765,Red Hot Chili Peppers,Under The Bridge
30804,Ed Sheeran,New Man
30809,Ed Sheeran,Galway Girl
31621,Empire of the Sun,High And Low
